In [ ]:
import pandas as pd
import shutil
import os
import numpy as np
import matplotlib.pyplot as plt
import onekey_algo.custom.components as okcomp
from onekey_algo import get_param_in_cwd

plt.rcParams['figure.dpi'] = 300
model_names = ['Clinical', 'DLRadiomics', 'Pathomics', 'Nomogram']
# 获取配置
task = get_param_in_cwd('task_column') or 'label'
bst_model = get_param_in_cwd('sel_model') or 'LR'
labelf = 'clinic_sel.csv'
group_info = get_param_in_cwd('dataset_column') or 'group'

# 读取label文件。
labels = [task]
label_data_ = pd.read_csv(labelf)
# label_data_['ID'] = label_data_['ID'].map(lambda x: f"{x}v.nii.gz")
# label_data_ = label_data_[['ID', group_info, task]]
label_data_ = label_data_.dropna(axis=0)

ids = label_data_['ID']
print(label_data_.columns)
label_data = label_data_[['ID'] + labels]

label_data

# 训练集-汇总

In [ ]:
import pandas as pd
from onekey_algo.custom.components.comp1 import normalize_df
subset = 'train'
Clinic_results = pd.merge(pd.read_csv(f'./results/Clinic_RandomForest_{subset}.csv', header=0), label_data, on='ID', how='inner')
DLR_results = pd.merge(pd.read_csv(f'./results/Rad_3DL_ShuffleNet_{subset}.csv', header=0), label_data, on='ID', how='inner')
Path_results = pd.merge(pd.read_csv(f'./Pathology/results/Path_XGBoost_{subset}.csv', header=0, converters={'ID': lambda x: f"{x}.nii.gz"}), 
                        label_data, on='ID', how='inner')
ALL_results = pd.merge(pd.merge(Clinic_results, DLR_results, on='ID', how='inner'), 
                                Path_results, on='ID', how='inner')
ALL_results.columns = ['ID', '-0', model_names[0], task, 
                       '-00', model_names[1], '-l',
                       '-000', model_names[2], '-ll',]

# ALL_results = normalize_df(ALL_results, method='minmax', not_norm=['ID', 'label'])
Clinic = pd.read_csv('clinic_sel.csv')[['ID', 'Sex', 'Smoking_History']]
cnames = list(Clinic.columns[1:])
ALL_results = pd.merge(ALL_results, Clinic, on='ID', how='inner')
ALL_results = ALL_results.dropna(axis=1)
train_ids = ALL_results[['ID']]
ALL_results

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from onekey_algo.custom.components import metrics

model = LogisticRegression(random_state=0)
# model = SVC(probability=True, random_state=0)
data_x = ALL_results[cnames + model_names[1:-1]]
data_y = ALL_results[task]
from sklearn.utils import shuffle
xxxx, yyyy = shuffle(data_x, data_y)
model.fit(data_x, data_y)
results = model.predict_proba(data_x)
results = pd.DataFrame(results, index=ALL_results['ID'], columns=[f'{task}-0', f'{task}-1']).reset_index()
results.to_csv(f'./results/Nomo_{subset}.csv', index=False, header=True)
pd.DataFrame([metrics.analysis_pred_binary(ALL_results[task], results[f'{task}-1'])], 
                  columns=['acc', 'auc', '95%CI', 'Sensitivity', 'Specificity', 'PPV', 'NPV', 'Precision', 'Recall', 'F1', 'Threshold'])

In [ ]:
print(model.coef_, model.intercept_)

In [ ]:
results.columns = ['ID', '-', model_names[-1]]
ALL_results = pd.merge(ALL_results, results, on='ID', how='inner')
gt = [np.array(ALL_results[labels]) for d in model_names]
pred_train = [np.array(ALL_results[mn]) for mn in model_names]
okcomp.comp1.draw_roc(gt, pred_train, labels=model_names, title=f"Model AUC")
plt.savefig(f'img/{subset}_auc.svg')

In [ ]:
from onekey_algo.custom.components.metrics import analysis_pred_binary
metric = []
for mname, y, score in zip(model_names, gt, pred_train):
    # 计算验证集指标
    acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres = analysis_pred_binary(y, score)
    ci = f"{ci[0]:.4f} - {ci[1]:.4f}"
    metric.append((mname, acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres, f"Train"))
pd.DataFrame(metric, index=None, columns=['Signature', 'Accuracy', 'AUC', '95% CI', 'Sensitivity', 'Specificity', 
                                          'PPV', 'NPV', 'Precision', 'Recall', 'F1','Threshold', 'Cohort'])

In [ ]:
from onekey_algo.custom.components.delong import delong_roc_test
from onekey_algo.custom.components.comp1 import draw_matrix

delong = []
delong_columns = []
this_delong = []
plt.figure(figsize=(8, 6))
cm = np.zeros((len(model_names), len(model_names)))
for i, mni in enumerate(model_names):
    for j, mnj in enumerate(model_names):
        if i <= j:
            cm[i][j] = np.nan
        else:
            cm[i][j] = delong_roc_test(ALL_results[task], ALL_results[mni], ALL_results[mnj])[0][0]
cm = pd.DataFrame(cm[1:, :-1], index=model_names[1:], columns=model_names[:-1])
draw_matrix(cm, annot=True, cmap='jet_r', cbar=True)
plt.title(f'Cohort {subset} Delong')
plt.savefig(f'img/delong_each_cohort_{subset}.svg', bbox_inches = 'tight')
plt.show()

In [ ]:
from onekey_algo.custom.components.comp1 import plot_DCA
plot_DCA([ALL_results[model_name] for model_name in model_names], 
         ALL_results[task], title=f'Cohort {subset} DCA', labels=model_names, y_min=-0.15)
plt.savefig(f'img/{subset}_dca.svg')

In [ ]:
from onekey_algo.custom.components.comp1 import draw_calibration
draw_calibration(pred_scores=pred_train, n_bins=5, remap=True,
                 y_test=gt, model_names=model_names)
plt.savefig(f'img/{subset}_cali.svg')

In [ ]:
from onekey_algo.custom.components import stats

hosmer = []
hosmer.append([stats.hosmer_lemeshow_test(y_true, y_pred, bins=10, remap=False) 
              for fn, y_true, y_pred in zip(model_names, gt, pred_train)])
pd.DataFrame(hosmer, columns=model_names)

# 训练Cox模型

In [ ]:
from onekey_algo.custom.components import nomogram
import shutil

ALL_results = ALL_results.round(decimals=2)
ALL_results = normalize_df(ALL_results, method='minmax', not_norm=['ID', 'label'])
nomogram.risk_nomogram(ALL_results, result=task, columns=cnames + model_names[1:-1], 
                       width=6000, height=3000,
                      x_range='0.01,0.25,0.50,0.75,0.99', save_name='nomogram_risk.png')

# 验证集-汇总

In [ ]:
import pandas as pd

subset = 'test'
Clinic_results = pd.merge(pd.read_csv(f'./results/Clinic_RandomForest_{subset}.csv', header=0), label_data, on='ID', how='inner')
DLR_results = pd.merge(pd.read_csv(f'./results/Rad_3DL_ShuffleNet_{subset}.csv', header=0), label_data, on='ID', how='inner')
Path_results = pd.merge(pd.read_csv(f'./Pathology/results/Path_XGBoost_{subset}.csv', header=0, converters={'ID': lambda x: f"{x}.nii.gz"}), 
                        label_data, on='ID', how='inner')
ALL_results = pd.merge(pd.merge(Clinic_results, DLR_results, on='ID', how='inner'), 
                                Path_results, on='ID', how='inner')
ALL_results.columns = ['ID', '-0', model_names[0], task, 
                       '-00', model_names[1], '-l',
                       '-000', model_names[2], '-ll',]

# ALL_results = normalize_df(ALL_results, method='minmax', not_norm=['ID', 'label'])
Clinic = pd.read_csv('clinic_sel.csv')[['ID', 'Sex', 'Smoking_History']]
cnames = list(Clinic.columns[1:])
ALL_results = pd.merge(ALL_results, Clinic, on='ID', how='inner')
ALL_results = ALL_results.dropna(axis=1)
test_ids = ALL_results[['ID']]
ALL_results

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from onekey_algo.custom.components import metrics

data_x = ALL_results[cnames + model_names[1:-1]]
data_y = ALL_results[task]
model = LogisticRegression(random_state=0, penalty='none')
# model = SVC(probability=True, random_state=0)
model.fit(data_x, data_y)
results = model.predict_proba(data_x)
results = pd.DataFrame(results, index=ALL_results['ID'], columns=[f'{task}-0', f'{task}-1']).reset_index()
results.to_csv(f'./results/Nomo_{subset}.csv', index=False, header=True)
pd.DataFrame([metrics.analysis_pred_binary(ALL_results[task], results[f'{task}-1'])], 
                  columns=['acc', 'auc', '95%CI', 'Sensitivity', 'Specificity', 'PPV', 'NPV', 'Precision', 'Recall', 'F1', 'Threshold'])

In [ ]:
pred_column = [f'{task}-0', f'{task}-1']
Nomo_results = pd.read_csv(f'./results/Nomo_{subset}.csv', header=0)
Nomo_results.columns = ['ID', 'label-9', model_names[-1]]
ALL_results = pd.merge(ALL_results, Nomo_results, on='ID', how='inner')
gt = [np.array(ALL_results[task]) for _ in model_names]
pred_train = [np.array(ALL_results[d]) for d in model_names]
okcomp.comp1.draw_roc(gt, pred_train, labels=model_names, title=f'Cohort {subset} ROC')
plt.savefig(f'img/{subset}_auc.svg')

In [ ]:
from onekey_algo.custom.components.metrics import analysis_pred_binary
for mname, y, score in zip(model_names, gt, pred_train):
    # 计算验证集指标
    acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres = analysis_pred_binary(y, score)
    ci = f"{ci[0]:.4f} - {ci[1]:.4f}"
    metric.append((mname, acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres, f"Test"))
pd.DataFrame(metric, index=None, columns=['Signature', 'Accuracy', 'AUC', '95% CI',
                                          'Sensitivity', 'Specificity', 'PPV', 'NPV', 'Precision', 'Recall', 'F1',
                                          'Threshold', 'Cohort'])

In [ ]:
delong = []
delong_columns = []
this_delong = []
plt.figure(figsize=(8, 6))
cm = np.zeros((len(model_names), len(model_names)))
for i, mni in enumerate(model_names):
    for j, mnj in enumerate(model_names):
        if i <= j:
            cm[i][j] = np.nan
        else:
            cm[i][j] = delong_roc_test(ALL_results[task], ALL_results[mni], ALL_results[mnj])[0][0]
cm = pd.DataFrame(cm[1:, :-1], index=model_names[1:], columns=model_names[:-1])
draw_matrix(cm, annot=True, cmap='jet_r', cbar=True)
plt.title(f'Cohort {subset} Delong')
plt.savefig(f'img/delong_each_cohort_{subset}.svg', bbox_inches = 'tight')

In [ ]:
from onekey_algo.custom.components.comp1 import plot_DCA
plot_DCA([ALL_results[model_name] for model_name in model_names], 
         ALL_results[task], title=f'Cohort {subset} DCA', labels=model_names, y_min=-0.15, remap=False)
plt.savefig(f'img/{subset}_dca.svg')

In [ ]:
from onekey_algo.custom.components.comp1 import draw_calibration
draw_calibration(pred_scores=pred_train, n_bins=5, remap=True,# smooth=True,
                 y_test=gt, model_names=model_names)
plt.savefig(f'img/{subset}_cali.svg')

In [ ]:
from onekey_algo.custom.components import stats

hosmer.append([stats.hosmer_lemeshow_test(y_true, y_pred, bins=5, remap=False) 
              for fn, y_true, y_pred in zip(model_names, gt, pred_train)])
pd.concat([pd.DataFrame(hosmer, columns=model_names), pd.DataFrame(['Train', 'Test'], columns=['Cohort'])], axis=1)

In [ ]:
import venn

venn_labels = venn.generate_petal_labels([set(pd.read_csv('group.csv')['ID']),
                                          set(pd.read_csv('Pathology/group.csv', converters={'ID': lambda x: f"{x}.nii.gz"})['ID']),
                                          set(train_ids['ID'])])
fig, ax = venn.venn2(venn_labels, names=['Radiomics', 'Pathomics', 'Train'], figsize=(10, 10))
fig.savefig('img/venn_samples.svg', bbox_inches='tight')
fig.show()

In [ ]:
train_ids['group'] = 'train'
test_ids['group'] = 'test'

pd.concat([train_ids, test_ids], axis=0).to_csv('data/joinit_ids.csv', index=False)